In [ ]:
# Práctica 8: ML US
# LIBRERIAS NECESARIAS (si os hacen falta más, debéis añadirlas)

import keras
import numpy as np
import os
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from UNET import *  

In [ ]:
# Folder with files path:
folder = os.getcwd() + '/benign'
print(folder)
# Alphabetically ordered list:
os.chdir(folder)
content = os.listdir(folder)
content.sort()
os.chdir('..')

# Remove .DS_Store file if exists:
if content[0] == '.DS_Store':
    content.pop(0)
    

In [ ]:
# Lista de imágenes y de máscaras
listImages = []
for im in content:
    listImages.append(im[:-4])
    
listImages = list(set(listImages))
listImages.sort()

listMasks = []
idx = []
for k1 in range(len(listImages)):
    if listImages[k1][-4:] == 'mask':
        listMasks.append(listImages[k1])
        idx.append(k1)
        
    if listImages[k1][-6:-2] == 'mask':
        listMasks.append(listImages[k1])
        idx.append(k1)

listImages = [v for i, v in enumerate(listImages) if i not in idx]

In [ ]:
# Retrieve images -> X_train_full 
X_train_full=[]

# Retrieve masks -> Y_train_full 
Y_train_full=[]

# Sin data augmentation, primero separaré el test
for k1 in listImages: # k1 -> Index for list of images
    Img = img_to_array(load_img(os.getcwd()+'/benign/'+k1+'.png', target_size=(400,400), color_mode='grayscale'))
    X_train_full.append(np.array(Img))

for k1 in range(len(listMasks)):
    mask = img_to_array(load_img(os.getcwd()+'/benign/'+listMasks[k1]+'.png', target_size=(400,400), color_mode='grayscale'))
    mask = mask > 0
    
    if listMasks[k1][-6:-2] == 'mask':
        Y_train_full[-1] = Y_train_full[-1] | mask
    
    else:
        Y_train_full.append(mask)       
        

In [ ]:
# Cojo para test las 50 primeras imágenes y las quito de los conjuntos
X_test = X_train_full[0:50]
X_train_full[0:50] = []

Y_test = Y_train_full[0:50]
Y_train_full[0:50] = []

In [ ]:
# Hago data augmentation de las imágenes
X_dA_train_full = np.zeros((len(X_train_full)*4,400,400,1))
index = list(range(0,len(X_dA_train_full)))
indexImg = index[0::4]

for k1 in range(len(X_train_full)):
    X_dA_train_full[indexImg[k1]] = X_train_full[k1]
    X_dA_train_full[indexImg[k1]+1] = np.flip(X_train_full[k1], (0, 2))
    X_dA_train_full[indexImg[k1]+2] = np.flip(X_train_full[k1], (1, 2))
    X_dA_train_full[indexImg[k1]+3] = np.flip(X_train_full[k1], (1, 0))


In [ ]:
# Hago data augmentation de las máscaras
Y_dA_train_full = np.zeros((len(Y_train_full)*4,400,400,1))
index = list(range(0,len(Y_dA_train_full)))
indexImg = index[0::4]

for k1 in range(len(Y_train_full)):
    Y_dA_train_full[indexImg[k1]] = Y_train_full[k1]
    Y_dA_train_full[indexImg[k1]+1] = np.flip(Y_train_full[k1], (0, 2))
    Y_dA_train_full[indexImg[k1]+2] = np.flip(Y_train_full[k1], (1, 2))
    Y_dA_train_full[indexImg[k1]+3] = np.flip(Y_train_full[k1], (1, 0))


In [ ]:
# Separar conjuntos de entrenamiento, validacion y test

# Normalizamos las imágenes de test
for k1 in range(len(X_test)):

    cropnorm = np.array(X_test[k1], dtype=np.float32)
    
    # Normalizamos para tener la imagen en el rango [0,1]
    cropnorm = cropnorm - cropnorm.min() 
    cropnorm = cropnorm / cropnorm.max()
    
    # Almacenamos el crop normaizado
    X_test[k1] = np.reshape(cropnorm,(400,400,1))
    
X_test = np.array(X_test, dtype='float32'); Y_test = np.array(Y_test, dtype='float32')


# Normalizamos las imágenes de entrenamiento
for k1 in range(len(X_dA_train_full)):

    cropnorm = np.array(X_dA_train_full[k1], dtype=np.float32)
    
    # Normalizamos para tener la imagen en el rango [0,1]
    cropnorm = cropnorm - cropnorm.min() 
    cropnorm = cropnorm / cropnorm.max()
    
    # Almacenamos el crop normaizado
    X_dA_train_full[k1] = np.reshape(cropnorm,(400,400,1))

np.random.seed(71)
idxTrain = np.random.permutation(len(X_dA_train_full))
Xt1 = np.array(X_dA_train_full, dtype=np.float32)[idxTrain,:,:,:]
Yt1 = np.array(Y_dA_train_full, dtype=np.float32)[idxTrain,:]
    
th1 = int(.75*(len(Yt1)))

# Generate train, validation and test sets:
X_train = np.array(Xt1[0:th1], dtype=np.float32)
X_valid = np.array(Xt1[th1+1:len(Xt1)], dtype=np.float32)
Y_train = np.array(Yt1[0:th1], dtype=np.float32)
Y_valid = np.array(Yt1[th1+1:len(Yt1)], dtype=np.float32)

del Xt1, Yt1, X_train_full, X_dA_train_full, Y_dA_train_full


In [ ]:
# Get the UNET model you want:
input_img = Input((400, 400, 1), name='img')
    
# UNET MODELS:
model = get_unet6(input_img, n_filters=14, dropout= 0.25, batchnorm=True)
model.compile(optimizer = Adam(learning_rate = 1e-3), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()


In [ ]:
# Training...
Save = True
if Save:
    results = model.fit(X_train, Y_train, epochs=10, validation_data=(X_valid, Y_valid))

# Representation of model accuracy and model loss
#Learning_results(results)

In [ ]:
# Guardamos pesos o los cargamos
if Save:
    model.save_weights('UNET.h5')
else:
    model.load_weights('UNET.h5')

In [ ]:
# Test y comprobacion
for k1 in range(len(X_test)):
    plt.figure(figsize=(400,400))
    fig, (ax1, ax2, ax3) = plt.subplots(1,3)
    ax1.imshow(X_test[k1].reshape((400,400)),cmap='gray')
    pred = model.predict(X_test[k1].reshape((1,400,400,1)))
    ax2.imshow(pred.reshape((400,400)), cmap='gray')
    ax3.imshow(Y_test[k1], cmap='gray')

### Cálculo coeficiente DICE

In [ ]:
DICE = dice_coef(Y_test,model.predict(X_test))
print(np.float32(DICE))

### Cálculo coeficiente JACCARD

In [ ]:
JACCARD = jaccard_coef(Y_test,model.predict(X_test))
print(np.float32(JACCARD))

### Cálculo métricas

In [ ]:
# Poner bien el nombre a las variables y eso
pred = model.predict(X_test)

# Umbralizamos la prediccion de la red con un umbral muy bajo
pred_umb = np.zeros(pred.shape)
pred_umb[np.where(pred>0.3)] = 1

(VP,VN,FP,FN) = calcula_valores(pred_umb,Y_test)

Acuraccy = (VP+VN)/(VP+FP+FN+VN)
Sensibilidad = VP/(VP+FN)
Especificidad = VN/(VN+FP)
VPP = VP/(VP+FP) # Valor predictivo positivo
VPN = VN/(VN + FN) # Valor predictivo negativo

print(Acuraccy)
print(Sensibilidad)
print(Especificidad)
print(VPP)
print(VPN)